In [2]:
# load binary file
import numpy as np
import time
import suite2p
from suite2p.run_s2p import run_s2p
import imp
imp.reload(suite2p)

def tic():
    return time.time()
def toc(i0):
    return time.time() - i0

ops = {
    'nplanes': 1,
    'diameter': 12,
    'tau': 2.,
    'fs': 2.5,
    'data_path': ['C:/Drive/suite2python/data1/'],
    #'data_path': ['C:/Users/carse/github/tiffs/'],
    #'save_path0': 'C:/',
    #'data_path': 'C:\Drive\suite2python\data',
    'subfolders': ['1', '2', '5'],
    'num_workers': 0,
    'num_workers_roi': -1,
    'reg_tif':False,
    'max_iterations':10,    
      }


# copy tiff to a binary
ops1 = run_s2p(ops)
#ops1 = register.tiff_to_binary(ops)

Found 50 tifs
time 21.5085. Wrote tifs to binaries for 1 planes
computed reference frame for registration
registered 4000/10000 frames in time 173.65
registered 8000/10000 frames in time 345.02
time 514.1733. Registration complete
SVD computed in 35.63 sec
cells: 215, cost: 0.6009, time: 65.1563
cells: 369, cost: 0.5893, time: 90.0053
cells: 460, cost: 0.5839, time: 117.8260
cells: 506, cost: 0.5815, time: 145.5998
cells: 520, cost: 0.5808, time: 172.8623
cells: 520, cost: 515882.9062, time: 211.0097
cells: 520, cost: 509372.9688, time: 230.5717
cells: 520, cost: 507856.4375, time: 250.7293
time 256.9247. Found 520 ROIs
extracted 0/10000 frames in 3.71 sec
extracted 5000/10000 frames in 22.70 sec
extracted 10000/10000 frames in 38.51 sec
time 304.0878. Extracted fluorescence from 520 ROIs
time 321.4638. Detected spikes in 520 ROIs
results saved to C:/Drive/suite2python/data1/suite2p\plane0
finished all tasks in 835.9830 sec
